In [79]:
import rdflib
import html
g = rdflib.Graph().parse('./fb15k-wikitop2021-yago-facts.nt', format='ntriples')
g += rdflib.Graph().parse('./fb15k-wikitop2021-yago-full-types.nt', format='ntriples')

from rdflib.namespace import RDF
for line in open('fb15k-wikitop2021.tsv').readlines()[1:]:
    subject = rdflib.URIRef( line.split('\t')[6][1:-1] ) # take the URI in column 6
    g.add( (subject, RDF.type, rdflib.URIRef('http://example.com/popularEntity')) )
    
g.remove((None, rdflib.URIRef('http://schema.org/image') , None))
g.remove((None, rdflib.URIRef('http://schema.org/url') , None))
g.serialize(destination='20q-updated.ttl', format='turtle')

<Graph identifier=N147c964a488c4db5beacf00d21d6d2c0 (<class 'rdflib.graph.Graph'>)>

In [77]:
r = g.query("""
select *  where 
{
    ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    ?s <http://schema.org/url> ?o .
} 
""")
for b in r.bindings:
    print(*[f'{x}' for x in b.values()])

## Code

In [1]:
%load_ext ipython_sparql_pandas

In [80]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper('http://localhost:7200/repositories/20qs-updated')
sparql.setReturnFormat(JSON)

In [81]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o']["value"])
        listOfPred.append(attr['p']["value"])
    return(listOfObj,listOfPred)

"""
def generateQuestion(listOfPred, listOfObj, index):
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]} ?' ).startswith('y')
    if userAnswer:
        PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.' )   
    else:
        NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}' )   
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)"""


def generateQuestion(listOfPred, listOfObj, index):
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        generateQuestion(listOfPred, listOfObj, index+1)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>')
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index].split("/")[-1]} {listOfObj[index].split("/")[-1]}?' )
    
    if userAnswer.startswith('y'):
        PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.')    
        
    elif userAnswer.startswith('n'):
        NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}')   
        
    else:
        generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)


  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            group by ?p ?o 
            ORDER BY ABS( {left} - ?count )
            Limit 10
            
            """) 
        
    return FilterQuestion

In [82]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return len(qres["results"]["bindings"])

In [83]:
# resets are variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s ?p ?o .
        }} 
        group by ?p ?o 
        ORDER BY ABS( %s - ?count )
        limit 10
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [84]:
#without printing results
left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
i=0
while i < 20:
   
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    if left<=1:
        break
    print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0)
    left = numberleft(PosFilters, NegFilters)
    query = updateQuery(left/2,PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    i += 1

if i != 20:
    finalResult= f"""
                select distinct ?s
                where 
                {{

                {PosFilters}
                ?s ?p ?o
                {NegFilters}
                }} """
    sparql.setQuery(finalResult)
    finalRes = sparql.queryAndConvert()
    for attr in finalRes["results"]["bindings"]:
        print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10]) 
else: 
    print('lost')

Question: 1


Does the thing you are looking for have the attribute: nationality United_States? y


Question: 2


Does the thing you are looking for have the attribute: hasOccupation Film_producer? y


Question: 3


...

Question: 5


Does the thing you are looking for have the attribute: award Theatre_World_Award? y


Question: 6


Does the thing you are looking for have the attribute: birthPlace Newton,_Massachusetts? y


Your Answer is: John_Krasinski


In [85]:
# with count print
left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
i=0
while i < 20:
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    if left<=1:
        break
    print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0)
    left = numberleft(PosFilters, NegFilters)
    query = updateQuery(left/2,PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    for r in qres["results"]["bindings"]:
        print(*[f'{x.split("/")[-1][:40]:40s}' for x in [r['count']["value"],r['p']["value"],r['o']["value"]]])
    i+=1

if i != 20:
    finalResult= f"""
                select distinct ?s
                where 
                {{

                {PosFilters}
                ?s ?p ?o
                {NegFilters}
                }} """
    sparql.setQuery(finalResult)
    finalRes = sparql.queryAndConvert()
    for attr in finalRes["results"]["bindings"]:
        print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10]) 
else:
    print('lost')

Question: 1


Does the thing you are looking for have the attribute: nationality United_States? y


44                                       hasOccupation                            Film_producer                           
34                                       hasOccupation                            Screenwriter                            
61                                       hasOccupation                            Actor                                   
28                                       memberOf                                 Democratic_Party_(United_States)        
26                                       hasOccupation                            Film_director                           
18                                       hasOccupation                            Stage_actor                             
18                                       homeLocation                             Los_Angeles                             
13                                       hasOccupation                            Songwriter                              
13              

Does the thing you are looking for have the attribute: hasOccupation Film_producer? y


22                                       hasOccupation                            Film_director                           
25                                       hasOccupation                            Screenwriter                            
13                                       memberOf                                 Democratic_Party_(United_States)        
13                                       homeLocation                             Los_Angeles                             
9                                        hasOccupation                            Stage_actor                             
37                                       hasOccupation                            Actor                                   
6                                        hasOccupation                            Character_actor                         
5                                        hasOccupation                            Singer-songwriter                       
5               

...

3                                        award                                    Theatre_World_Award                     
3                                        homeLocation                             Los_Angeles                             
4                                        hasOccupation                            Character_actor                         
2                                        givenName                                John_(given_name)                       
5                                        hasOccupation                            Screenwriter                            
2                                        hasOccupation                            Stage_actor                             
2                                        award                                    National_Society_of_Film_Critics_Award_f
2                                        award                                    New_York_Film_Critics_Circle_Award_for_B
2               

Does the thing you are looking for have the attribute: award Theatre_World_Award? y


1                                        birthPlace                               Newton,_Massachusetts                   
1                                        spouse                                   Emily_Blunt                             
1                                        givenName                                John_(given_name)                       
1                                        alumniOf                                 Newton_South_High_School                
1                                        alumniOf                                 Brown_University                        
1                                        birthDate                                1979-10-20                              
2                                        hasOccupation                            Screenwriter                            
2                                        hasOccupation                            Stage_actor                             
1               

Does the thing you are looking for have the attribute: birthPlace Newton,_Massachusetts? y


1                                        22-rdf-syntax-ns#type                    Human                                   
1                                        22-rdf-syntax-ns#type                    popularEntity                           
1                                        nationality                              United_States                           
1                                        birthPlace                               Newton,_Massachusetts                   
1                                        award                                    Theatre_World_Award                     
1                                        spouse                                   Emily_Blunt                             
1                                        givenName                                John_(given_name)                       
1                                        alumniOf                                 Newton_South_High_School                
1               

In [40]:
# with guess print
def guess(PosFilters, NegFilters):
    FilterQuestion =  f"""
            select ?s  where 
            {{
            {PosFilters}
             ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters}
            }} 
            """
        
    return FilterQuestion

i=0
left, qres, PosAttr, NegAttr, AttrHistory = game_reset()
while i < 20:
   
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    if left<=1:
        break
    print('Question: %s' %(i+1))
    PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0)
    left = numberleft(PosFilters, NegFilters)
    query = updateQuery(left/2, PosFilters, NegFilters)
    sparql.setQuery(query)
    qres = sparql.queryAndConvert()
    guesses = guess(PosFilters, NegFilters)
    sparql.setQuery(guesses)
    subjectsleft = sparql.queryAndConvert()
    print('Possible Guesses:')
    for subject in subjectsleft['results']['bindings']:
        print(*[f'{x.split("/")[-1]}' for x in [subject['s']['value']]][0:10]) 
    print()
    i += 1
    
if i != 20:
    finalResult= f"""
                select distinct ?s
                where 
                {{

                {PosFilters}
                ?s ?p ?o
                {NegFilters}
                }} """
    sparql.setQuery(finalResult)
    finalRes = sparql.queryAndConvert()
    for attr in finalRes["results"]["bindings"]:
        print(*[f'Your Answer is: {x.split("/")[-1]}' for x in [attr['s']['value']]][0:10]) 

Question: 1
['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']


Does the thing you are looking for have the attribute: nationality United_States? y


Possible Guesses:
Kanye_West
Eminem
Jennifer_Aniston
George_W._Bush
Prince_(musician)
John_Krasinski
Val_Kilmer
Brad_Pitt
Barack_Obama
Bill_Clinton
Donald_Trump
Jimmy_Carter
Martin_Luther_King_Jr.
Ronald_Reagan
Matthew_Perry
Robert_Downey_Jr.
Marilyn_Manson
Ben_Affleck
Serena_Williams
Michael_Jackson
Nicolas_Cage
Johnny_Depp
Tobey_Maguire
Sandra_Bullock
Paul_Walker
Mark_Wahlberg
Alec_Baldwin
Stanley_Tucci
Frank_Sinatra
Salma_Hayek
Ryan_Reynolds
Snoop_Dogg
Ernest_Hemingway
Brittany_Murphy
Lady_Gaga
Robin_Williams
Will_Smith
Mila_Kunis
Quentin_Tarantino
Tom_Hanks
Arnold_Schwarzenegger
Clint_Eastwood
Julia_Roberts
Sylvester_Stallone
Judy_Garland
Jake_Gyllenhaal
Betty_White
Tom_Cruise
Jason_Sudeikis
Scarlett_Johansson
Elvis_Presley
Vin_Diesel
Angelina_Jolie
Britney_Spears
Emily_Blunt
Selena_Gomez
Blake_Lively
Marilyn_Monroe
Matthew_McConaughey
Dwayne_Johnson
Tupac_Shakur
Abraham_Lincoln
Macaulay_Culkin
Kristen_Stewart
Albert_Einstein
Nicki_Minaj
Emma_Stone
Olivia_Wilde
Aretha_Franklin
Jenn

Does the thing you are looking for have the attribute: hasOccupation Film_producer? y


Possible Guesses:
Eminem
Jennifer_Aniston
Prince_(musician)
John_Krasinski
Val_Kilmer
Brad_Pitt
Matthew_Perry
Robert_Downey_Jr.
Ben_Affleck
Nicolas_Cage
Johnny_Depp
Tobey_Maguire
Sandra_Bullock
Paul_Walker
Mark_Wahlberg
Alec_Baldwin
Stanley_Tucci
Frank_Sinatra
Salma_Hayek
Ryan_Reynolds
Snoop_Dogg
Robin_Williams
Will_Smith
Quentin_Tarantino
Tom_Hanks
Arnold_Schwarzenegger
Clint_Eastwood
Julia_Roberts
Sylvester_Stallone
Tom_Cruise
Vin_Diesel
Angelina_Jolie
Selena_Gomez
Marilyn_Monroe
Matthew_McConaughey
Dwayne_Johnson
Jennifer_Lawrence
Bruce_Lee
Chris_Evans_(actor)
Leonardo_DiCaprio
Jason_Momoa
Nicole_Kidman
Shaquille_O'Neal
Steve_Jobs

Question: 3
['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>', '<http://schema.org/nationality> <http://yago-knowledge.org/resource/United_States>', '<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Film_producer>']


...

Possible Guesses:
John_Krasinski
Brad_Pitt
Ben_Affleck
Nicolas_Cage
Johnny_Depp
Alec_Baldwin
Tom_Hanks

Question: 5
['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>', '<http://schema.org/nationality> <http://yago-knowledge.org/resource/United_States>', '<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Film_producer>', '<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Film_director>', '<http://schema.org/memberOf> <http://yago-knowledge.org/resource/Democratic_Party_(United_States)>']


Does the thing you are looking for have the attribute: award Theatre_World_Award? y


Possible Guesses:
John_Krasinski
Alec_Baldwin
Tom_Hanks

Question: 6
['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>', '<http://schema.org/nationality> <http://yago-knowledge.org/resource/United_States>', '<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Film_producer>', '<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Film_director>', '<http://schema.org/memberOf> <http://yago-knowledge.org/resource/Democratic_Party_(United_States)>', '<http://schema.org/award> <http://yago-knowledge.org/resource/Theatre_World_Award>']


Does the thing you are looking for have the attribute: birthPlace Newton,_Massachusetts? y


Possible Guesses:
John_Krasinski

Your Answer is: John_Krasinski
